In [7]:
import numpy as np
from scipy.stats import geom, multivariate_normal

In [25]:
n_x = 20
n_z = 20
#A = size = n_z, n_x
#b = size = n_z, 1

def q_phi(x):
    mu = A @ x + b
    covariance = 2 / 3 * np.eye(n_z)
    return multivariate_normal(mean=mu, cov=covariance)

def p_theta(theta):
    mu = theta
    covariance = 2 * np.eye(n_x)
    return multivariate_normal(mean=mu, cov=covariance)


p = geom
r = 0.6

def logmeanexp(data, axis=None):
    max_val = np.max(data, axis=axis)
    return max_val + np.log(np.mean(np.exp(data - max_val), axis=axis))

In [26]:
K = p.rvs(r)
size = 2 ** (K + 1)
print(K)

2


In [29]:
theta_0 = [8] * n_x
theta_0 = np.array(theta_0)
X = p_theta(theta_0).rvs(size=size)
print(X[:2])
print(X.shape)

[[ 9.3202681   8.81218288  5.12285399  7.31247107  8.93098337  8.95831285
   5.73910914  5.65676393  6.65925569 10.30061467  6.78588316  7.01575551
  10.63648887  9.39163455  6.57905734  8.15018817  9.89773672  8.85457616
   6.68365735  9.08616354]
 [ 9.41890537  8.67855439  7.01372266  6.2776782   8.87902822  8.57002038
   7.70731239  7.63449764 10.0148274   7.38254875  7.764777    9.48985394
   8.8937638   6.148231    7.22821369 10.30664621  8.12328976  7.73075144
   8.85203508  8.99445784]]
(8, 20)


In [ ]:
Z = [q_phi(x).rvs() for x in X]
Z_E, Z_O = Z[::2], Z[1::2]
weights = 
log_weights = log(weights)
I_0 = np.mean(log_weights)
